In [1]:
COST_INDEX = 0.07
SELL_TAX = 0.036
LOCATION = 60003760
REGION = 10000002
DURATION_MODIFIER = (1 - 0.32) * (1 - 0.15)
BP_LIST = """Capital Quantum Microprocessor Blueprint	10	20	-1	Advanced Capital Construction Components
Capital Crystalline Carbonide Armor Plate Blueprint	0	0	-1	Advanced Capital Construction Components
Capital Compact Pb-Acid Cap Battery Blueprint	0	0	-1	Capacitor Battery
Cap Booster 150 Blueprint	10	20	-1	Capacitor Booster Charge
Hornet I Blueprint	10	0	-1	Combat Drone
Skirmish Command Burst I Blueprint	0	0	-1	Command Burst
Shield Command Burst I Blueprint	0	0	-1	Command Burst
Sustained Shield Emitter Blueprint	10	20	-1	Construction Components
Minmatar Control Tower Small Blueprint	0	0	-1	Control Tower
Amarr Control Tower Small Blueprint	0	0	-1	Control Tower
Gallente Control Tower Small Blueprint	0	0	-1	Control Tower
Corporate Hangar Array Blueprint	7	0	-1	Corporate Hangar Array
Capital Gremlin Compact Energy Neutralizer Blueprint	0	0	-1	Energy Neutralizer
Capital Infectious Scoped Energy Neutralizer Blueprint	5	0	-1	Energy Neutralizer
Hermes Compact Fighter Support Unit Blueprint	0	0	-1	Fighter Support Unit
Warfare Computation Core Blueprint	10	20	-1	Hybrid Tech Components
Optimized Nano-Engines Blueprint	10	20	-1	Hybrid Tech Components
Metallofullerene Plating Blueprint	0	0	-1	Hybrid Tech Components
Superconducting Ladar Amplifier Blueprint	10	20	-1	Hybrid Tech Components
Reconfigured Subspace Calibrator Blueprint	10	20	-1	Hybrid Tech Components
Light Hull Maintenance Bot I Blueprint	10	20	-1	Logistic Drone
Civilian Mining Drone Blueprint	10	0	-1	Mining Drone
EMP M Blueprint	0	0	-1	Projectile Ammo
Carbonized Lead M Blueprint	0	0	-1	Projectile Ammo
Phased Plasma M Blueprint	0	0	-1	Projectile Ammo
Proton M Blueprint	0	0	-1	Projectile Ammo
Titanium Sabot M Blueprint	0	0	-1	Projectile Ammo
Carbonized Lead S Blueprint	10	0	-1	Projectile Ammo
Depleted Uranium S Blueprint	0	0	-1	Projectile Ammo
EMP S Blueprint	0	0	-1	Projectile Ammo
Fusion S Blueprint	0	0	-1	Projectile Ammo
Nuclear S Blueprint	0	0	-1	Projectile Ammo
Phased Plasma S Blueprint	0	0	-1	Projectile Ammo
Proton S Blueprint	0	0	-1	Projectile Ammo
Titanium Sabot S Blueprint	10	0	-1	Projectile Ammo
50000MN Cold-Gas Enduring Microwarpdrive Blueprint	0	0	-1	Propulsion Module
10000MN Monopropellant Enduring Afterburner Blueprint	0	0	-1	Propulsion Module
10000MN Afterburner I Blueprint	0	0	-1	Propulsion Module
Capital Coaxial Compact Remote Armor Repairer Blueprint	0	10	-1	Remote Armor Repairer
Capital I-ax Enduring Remote Armor Repairer Blueprint	0	0	-1	Remote Armor Repairer
Medium Higgs Anchor I Blueprint	0	0	-1	Rig Anchor
Small Higgs Anchor I Blueprint	0	0	-1	Rig Anchor
Large Thermal Armor Reinforcer I Blueprint	8	0	-1	Rig Armor
Small Processor Overclocking Unit I Blueprint	10	20	-1	Rig Core
Small Command Processor I Blueprint	0	0	-1	Rig Core
Capital Inverted Signal Field Projector I Blueprint	0	0	-1	Rig Electronic Systems
Small Energy Locus Coordinator I Blueprint	0	0	-1	Rig Energy Weapon
Small Hyperspatial Velocity Optimizer I Blueprint	10	0	-1	Rig Navigation
Small Low Friction Nozzle Joints I Blueprint	10	0	-1	Rig Navigation
Medium Low Friction Nozzle Joints I Blueprint	0	0	-1	Rig Navigation
Medium Hyperspatial Velocity Optimizer I Blueprint	0	0	-1	Rig Navigation
Medium Warp Core Optimizer I Blueprint	0	0	-1	Rig Navigation
Medium Polycarbon Engine Housing I Blueprint	0	0	-1	Rig Navigation
Medium Engine Thermal Shielding I Blueprint	0	0	-1	Rig Navigation
Medium Dynamic Fuel Valve I Blueprint	0	0	-1	Rig Navigation
Medium Cargohold Optimization I Blueprint	0	0	-1	Rig Navigation
Medium Auxiliary Thrusters I Blueprint	0	0	-1	Rig Navigation
Small Warp Core Optimizer I Blueprint	0	0	-1	Rig Navigation
Small Polycarbon Engine Housing I Blueprint	0	0	-1	Rig Navigation
Small Engine Thermal Shielding I Blueprint	0	0	-1	Rig Navigation
Small Dynamic Fuel Valve I Blueprint	0	0	-1	Rig Navigation
Small Cargohold Optimization I Blueprint	0	0	-1	Rig Navigation
Small Auxiliary Thrusters I Blueprint	0	0	-1	Rig Navigation
Small Signal Focusing Kit I Blueprint	10	0	-1	Rig Scanning
Capital Core Defense Operational Solidifier I Blueprint	0	0	-1	Rig Shield
Large Core Defense Operational Solidifier I Blueprint	0	0	-1	Rig Shield
Scan Rangefinding Array I Blueprint	10	0	-1	Scanning Upgrade
Large Shield Extender I Blueprint	0	0	-1	Shield Extender
Ship Maintenance Array Blueprint	7	0	-1	Ship Maintenance Array
Caldari Shuttle Blueprint	7	10	-1	Shuttle
Ice Harvester I Blueprint	9	0	-1	Strip Miner
Standup Ladar ECM Script Blueprint	10	20	-1	Structure ECM script
R.A.M.- Ammunition Tech Blueprint	0	0	-1	Tool
R.A.M.- Armor/Hull Tech Blueprint	0	0	-1	Tool
R.A.M.- Electronics Blueprint	0	0	-1	Tool
R.A.M.- Energy Tech Blueprint	0	0	-1	Tool
R.A.M.- Robotics Blueprint	0	0	-1	Tool
R.A.M.- Shield Tech Blueprint	0	0	-1	Tool
R.A.M.- Starship Tech Blueprint	0	0	-1	Tool
R.A.M.- Weapon Tech Blueprint	0	0	-1	Tool
R.Db - Core Complexion Blueprint	0	0	-1	Tool
Optimal Range Disruption Script Blueprint	8	0	-1	Tracking Disruption Script
Nova XL Torpedo Blueprint	8	16	-1	XL Torpedo"""

In [2]:
from src.util import connect_to_db

conn = await connect_to_db()

bp_info = await conn.fetch('SELECT * FROM sde.blueprints')
bp_materials = await conn.fetch("SELECT * FROM sde.blueprint_materials WHERE activity = 'manufacturing'")
bp_products = await conn.fetch("SELECT * FROM sde.blueprint_products WHERE activity = 'manufacturing'")
bp_products = {bp['blueprint_type_id']: {'type_id':bp['product_type_id'], 'quantity': bp['quantity']} for bp in bp_products}

type_ids = dict(await conn.fetch('SELECT type_id, name FROM sde.type_ids'))
type_names = dict(await conn.fetch('SELECT name, type_id FROM sde.type_ids'))
type_published = dict(await conn.fetch('SELECT type_id, published FROM sde.type_ids'))

market_buy = dict(await conn.fetch(f'SELECT type_id, buy_max FROM market.aggregates WHERE location_id = {LOCATION}'))
market_sell = dict(await conn.fetch(f'SELECT type_id, sell_min FROM market.aggregates WHERE location_id = {LOCATION}'))
market_region_sell = dict(await conn.fetch(f'SELECT type_id, sell_min FROM market.aggregates WHERE region_id = {REGION}'))

blueprints = {}
for bp in bp_info:
    if type_published[bp['blueprint_type_id']] == False:
        continue
    if bp['blueprint_type_id'] not in bp_products:
        continue
    blueprints[bp['blueprint_type_id']] = {
        'name': type_ids[bp['blueprint_type_id']],
        'manufacture_time': bp['manufacturing_time'],
        'materials': [],
        'product': bp_products[bp['blueprint_type_id']],
    }
for bp_mat in bp_materials:
    if bp_mat['blueprint_type_id'] not in blueprints:
        continue
    blueprints[bp_mat['blueprint_type_id']]['materials'].append({
        'type_id': bp_mat['material_type_id'],
        'quantity': bp_mat['quantity'],
    })

In [3]:
import math
from decimal import Decimal

SELL_TAX = Decimal(SELL_TAX)
COST_INDEX = Decimal(COST_INDEX)

sell_price = {}
buy_price = {}
profit = {}
profit_per_day = {}
profitable_price = {}
bp_price = {}

owned_mat_eff = {}
owned_time_eff = {}
for bp_line in BP_LIST.split("\n"):
    bp_name, bp_mat_eff, bp_time_eff, _, _ = bp_line.split("\t")
    owned_mat_eff[type_names[bp_name]] = int(bp_mat_eff) / 100
    owned_time_eff[type_names[bp_name]] = int(bp_time_eff) / 100

for bp_id, bp in blueprints.items():
    if bp['product']['type_id'] not in market_buy or market_buy[bp['product']['type_id']] is None:
        continue
    if bp['product']['type_id'] not in market_sell or market_sell[bp['product']['type_id']] is None:
        continue
    
    bp_price[bp_id] = market_region_sell[bp_id] if bp_id in market_region_sell else 0
    bp_price[bp_id] = bp_price[bp_id] if bp_price[bp_id] is not None else 0
    
    buy_price[bp_id] = 0
    cant_buy = False
    for material in bp['materials']:
        if material['type_id'] not in market_sell or market_sell[material['type_id']] is None:
            cant_buy = True
            break
        buy_quantity = math.ceil(material['quantity'] * (1 - owned_mat_eff[bp_id] if bp_id in owned_mat_eff else 1))
        buy_price[bp_id] += market_sell[material['type_id']] * buy_quantity
        
    if cant_buy:
        continue
    
    profitable_price[bp_id] = buy_price[bp_id] / (1 - COST_INDEX) / (1 - SELL_TAX) / bp['product']['quantity']
    
    sell_price[bp_id] = market_buy[bp['product']['type_id']] * bp['product']['quantity']
    # sell_price[bp_id] = market_sell[bp['product']['type_id']] * bp['product']['quantity']
    profit[bp_id] = sell_price[bp_id] - buy_price[bp_id] - sell_price[bp_id] * SELL_TAX - buy_price[bp_id] * COST_INDEX
    profit_per_day[bp_id] = profit[bp_id] * Decimal(86400 / (bp['manufacture_time'] * DURATION_MODIFIER * (1 - owned_time_eff[bp_id] if bp_id in owned_time_eff else 1)))

print(f"{'Name':<64} {'Profit per day':>16} {'Profit':>16} {'Buy price':>16} {'Sell price':>16} {'Runs/BP':>8} {'Days/BP':>8}")
for bp_id, bp_profit in sorted(profit_per_day.items(), key=lambda item: item[1], reverse=True):
    if bp_price[bp_id] == 0:
        continue
    if bp_id in owned_mat_eff:
        continue
    print(f"{type_ids[bp_id]:64} {bp_profit:16,.2f} {profit[bp_id]:16,.2f} {buy_price[bp_id]:16,.2f} {sell_price[bp_id]:16,.2f} {bp_price[bp_id] / profit[bp_id]:8.2f} {bp_price[bp_id] / profit_per_day[bp_id]:8.2f}")
    

Name                                                               Profit per day           Profit        Buy price       Sell price  Runs/BP  Days/BP
Magnetometric-FTL Interlink Communicator Blueprint                 371,192,727.86    39,731,369.76   121,882,832.00   176,500,000.00    49.31     5.28
Radar-FTL Interlink Communicator Blueprint                         155,949,989.11    16,692,424.76   125,846,332.00   157,000,000.00   119.46    12.79
'Azmaru' Electromagnetic Disruptive Lance Blueprint                 54,161,979.24    26,088,020.00   231,394,000.00   283,900,000.00    19.91     9.59
Capital Asymmetric Enduring Remote Shield Booster Blueprint         31,103,229.34     7,490,694.40    38,046,080.00    50,000,000.00     6.54     1.58
Caldari Control Tower Medium Blueprint                              15,379,498.96     6,173,160.00   236,492,000.00   268,900,000.00    40.50    16.26
Personal Hangar Array Blueprint                                     13,111,092.85     1,052,52

In [4]:
print(f"{'Name':<64} {'Profit / day':>16} {'Profit / run':>16} {'Buy price':>16} {'Sell price':>16} {'Profitable price':>16}")
for bp_id, bp_profit in sorted(profit_per_day.items(), key=lambda item: item[1], reverse=True):
    if bp_id in owned_mat_eff:
        print(f"{type_ids[bp_id]:64} {bp_profit:16,.2f} {profit[bp_id]:16,.2f} {buy_price[bp_id]:16,.2f} {sell_price[bp_id]:16,.2f} {profitable_price[bp_id]:16,.2f}")

Name                                                                 Profit / day     Profit / run        Buy price       Sell price Profitable price
Ship Maintenance Array Blueprint                                    89,694,435.99     7,200,470.00    29,335,000.00    40,030,000.00    32,720,965.51
Hermes Compact Fighter Support Unit Blueprint                       77,183,958.48    18,588,470.00   109,299,000.00   140,600,000.00   121,914,736.98
EMP M Blueprint                                                      2,239,981.79         8,991.04         6,480.60        16,520.00            72.29
Optimal Range Disruption Script Blueprint                            2,198,691.16         4,412.65        19,660.70        26,400.00        21,930.02
Medium Hyperspatial Velocity Optimizer I Blueprint                   1,248,942.79        25,065.59       976,611.60     1,110,000.00     1,089,336.10
Shield Command Burst I Blueprint                                     1,172,357.44        28,234.27  